In [1]:
import pandas as pd
import numpy as np
import re 

In [2]:
Newlogs = pd.read_csv('data/16.04.csv')#need to change manually!!!!!
AllData = pd.read_csv('data/allCommentsData.csv')
UsersWithId = pd.read_csv('data/UsersWithId.csv')
last_update = pd.read_csv('data/lastUpdate_Comments.csv')
Hero  = 8
Champion  = 30
Guro = 50
weekNo = 'week5'#need to change menualy!!!!!

In [3]:
AllData

,Sum,Hero,Champion,Guro,User full name,week1_x,week1_y
0,5.0,NaN,NaN,NaN,Kobi Gal,4.0,1.0
1,2.0,NaN,NaN,NaN,אהוד מלול,2.0,NaN
2,2.0,NaN,NaN,NaN,אורי סלומון,2.0,NaN
3,1.0,NaN,NaN,NaN,אייל מאור,1.0,NaN
4,9.0,1.0,NaN,NaN,איתי אסרף,9.0,NaN
5,1.0,NaN,NaN,NaN,אלון שניידר,1.0,NaN
6,2.0,NaN,NaN,NaN,אלכסנדר גלר,2.0,NaN
7,1.0,NaN,NaN,NaN,בר אברהם,1.0,NaN
8,8.0,1.0,NaN,NaN,גיא אלוביץ,8.0,NaN
9,6.0,NaN,NaN,NaN,דור כהן,6.0,NaN


In [4]:
def Deletedcomments(data):
    Deletedcommentss = pd.DataFrame()
    for index, row in data.iterrows():
        if  (row['Component'] =='Post deleted'):
            CommentId = re.search('has deleted the post with id (.+?) in ', row['Event name'])
            if CommentId:
                found = CommentId.group(1)
                Deletedcommentss = Deletedcommentss.append({'CommentId': found},ignore_index=True)
    return Deletedcommentss

In [5]:
Deletedcomments = Deletedcomments(Newlogs)

In [6]:
def IsDeleted(row):
    CommentId = re.search('has created the post with id (.+?) in', row['Event name'])
    if CommentId:
        found = CommentId.group(1)
        return  pd.Series(Deletedcomments['CommentId']).str.contains(found, regex=False).any()
             

In [10]:
def calculateOutput(data) : # adding 1 if the row is a Comment 
#     last_update_Date = pd.to_datetime(last_update['last_Date'].iloc[0])
    for index, row in data.iterrows():
        name =  row['User full name']
        rowTime = pd.to_datetime(row['Time'].replace(',', ''))
#         if  np.logical_and(row['Component'] =='Discussion created' , rowTime > last_update_Date):
        if  (row['Component'] =='Post created'):
            if (not IsDeleted(row)):
                data.loc[index,weekNo] = 1  
            else : 
                data.loc[index,weekNo] = 0
        else :
            data.loc[index,weekNo] = 0 
    return data.reset_index()

In [11]:
UpdateData = calculateOutput(Newlogs)


In [15]:
# Pass the row elements as key value pairs to append() function 
# AllData = AllData.append({'User full name' : 'Daniel' } , ignore_index=True)
UpdateData[weekNo].sum()

92.0

In [99]:
def Summarizer(UpdateData) :
    sume = np.sum(UpdateData[weekNo]) 
    s2 = pd.Series(sume, name = weekNo)
    
    return pd.concat([s2], axis=1)

In [100]:
UpdateData = UpdateData.groupby('User full name').apply(lambda x: Summarizer(x)).reset_index()

In [101]:
UpdateData = UpdateData[UpdateData[weekNo] >0 ]
# UpdateData.to_csv('data/output_29_03_21_comments.csv')

In [102]:
UpdateData = UpdateData.drop(['level_1'], axis=1)

In [103]:
UpdateData

,User full name,week5
0,Kobi Gal,2.0
15,איתי אסרף,1.0
20,אלון מלאך,1.0
22,אלון שניידר,1.0
31,גיא אלוביץ,1.0
38,דור לוי,1.0
39,דור שרטו,1.0
47,דניאל דינשטיין,1.0
54,הראל שלום מושיוף,2.0
63,יהונתן אבקסיס,1.0


In [104]:
result = pd.merge(AllData, UpdateData, how='outer', on=['User full name'])

In [105]:
result['Sum']= result.drop(['Hero','Champion','Guro','Sum'], axis=1).sum(axis=1) #summing all weeks

In [106]:
def IssueBadges(result):
    IssueToUsers = pd.DataFrame()
    for index, row in result.iterrows():
        B1 = 0
        B2 = 0
        B3 = 0
        if  (np.logical_and(row['Sum'] >= Hero  , row.Hero != 1)): 
            result.loc[index,'Hero'] = 1 
            B1=1
        if  np.logical_and(row['Sum'] >= Champion , row.Champion != 1): 
            result.loc[index,'Champion'] = 1 
            B2=1
        if  np.logical_and(row['Sum'] >= Guro , row.Guro != 1): 
            result.loc[index,'Guro'] = 1 
            B3=1
        IssueToUsers =  IssueToUsers.append({'User full name': row['User full name'],'Hero' :B1,'Champion' :B2,'Guro' :B3}, ignore_index=True)
    return IssueToUsers

In [107]:
AllData = result 

In [108]:
ToIssueToday = IssueBadges(result)
ToIssueToday

,Champion,Guro,Hero,User full name
0,0.0,0.0,0.0,Kobi Gal
1,0.0,0.0,0.0,אלכסנדר גלר
2,0.0,0.0,0.0,דין דניאל עזרא
3,0.0,0.0,0.0,מור לוי
4,0.0,0.0,0.0,עמית אביטן
5,0.0,0.0,0.0,איתי אסרף
6,0.0,0.0,0.0,אלון מלאך
7,0.0,0.0,0.0,אלון שניידר
8,0.0,0.0,0.0,גיא אלוביץ
9,0.0,0.0,0.0,דור לוי


In [109]:
AllData

,Sum,Hero,Champion,Guro,User full name,week4,week5
0,3.0,NaN,NaN,NaN,Kobi Gal,1.0,2.0
1,2.0,NaN,NaN,NaN,אלכסנדר גלר,2.0,NaN
2,1.0,NaN,NaN,NaN,דין דניאל עזרא,1.0,NaN
3,1.0,NaN,NaN,NaN,מור לוי,1.0,NaN
4,7.0,NaN,NaN,NaN,עמית אביטן,3.0,4.0
5,1.0,NaN,NaN,NaN,איתי אסרף,NaN,1.0
6,1.0,NaN,NaN,NaN,אלון מלאך,NaN,1.0
7,1.0,NaN,NaN,NaN,אלון שניידר,NaN,1.0
8,1.0,NaN,NaN,NaN,גיא אלוביץ,NaN,1.0
9,1.0,NaN,NaN,NaN,דור לוי,NaN,1.0


In [110]:
UsersWithId = UsersWithId.rename(columns={UsersWithId.columns[0]: "User full name",UsersWithId.columns[1]: "id"})

In [111]:
ToIssueToday = pd.merge(ToIssueToday, UsersWithId, how='left', on=['User full name'])

In [112]:
ToIssueToday =  ToIssueToday[ToIssueToday['id'].notna()]

In [113]:
#updating the Last update date :
last_date = {'last_Date' : [np.max(pd.to_datetime(Newlogs['Time'].str.replace(',', '')))]}# converting str to timeStamp and takes the last date
last_date = pd.DataFrame(last_date)
last_date.to_csv('data/lastUpdate_Comments.csv',index=False)
result.to_csv('data/allCommentsData.csv',index=False)
ToIssueToday.to_csv('data/ToIssueToday_Comments.csv',index=False)